###WEEK 38 - (feed-forward net, random forest, and logistic regression)

In [ ]:
import numpy as np, pandas as pd, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack


In [34]:
#Loading data
langs  = ["ar","ko","te"]
splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df_train = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
df_val   = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])
df_train = df_train[df_train.lang.isin(langs)].reset_index(drop=True)
df_val   = df_val[df_val.lang.isin(langs)].reset_index(drop=True)

train_ds = Dataset.from_pandas(df_train[["lang","question","context","answerable","answer_start","answer"]], preserve_index=False)
val_ds   = Dataset.from_pandas(df_val[  ["lang","question","context","answerable","answer_start","answer"]], preserve_index=False)

In [ ]:
# Helpers / Metrics / Threshold

device = "cuda" if torch.cuda.is_available() else "cpu"

def to_numpy_labels(ds):
    return np.array([1 if bool(x) else 0 for x in ds["answerable"]], dtype=np.int64)

def label_stats(y, name=""):
    p = y.mean()
    print(f"[{name}] n={len(y)}  positives={y.sum()} ({p:.3f})  negatives={(1-p):.3f}")

def metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    acc = accuracy_score(y_true, y_pred)
    tpr = tp/(tp+fn) if (tp+fn)>0 else 0.0
    fpr = fp/(fp+tn) if (fp+tn)>0 else 0.0
    return acc, tpr, fpr

def pick_threshold(y_true, y_score, goal="f1"):
    #Default: choose threshold with maximum F1
    if goal == "f1":
        best_t, best_f1 = 0.5, -1.0
        for t in np.linspace(0.0, 1.0, 101):
            y_pred = (y_score >= t).astype(int)
            f1 = f1_score(y_true, y_pred, zero_division=0)
            if f1 > best_f1:
                best_f1, best_t = f1, t
        return best_t
    #Alternative: smallest FPR with TPR >= 0.9
    fpr, tpr, thr = roc_curve(y_true, y_score)
    mask = tpr >= 0.90
    return (thr[mask][np.argmin(fpr[mask])]) if mask.any() else 0.5

In [ ]:
#Feature Extraction

tok  = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
bert = AutoModel.from_pretrained("distilbert-base-multilingual-cased").to(device)
bert.eval()
print("hidden_size from config:", bert.config.hidden_size)

@torch.no_grad()
def mean_embed(texts, batch_size=16, max_length=256):
    vecs = []
    for i in range(0, len(texts), batch_size):
        enc = tok(texts[i:i+batch_size], padding=True, truncation=True,
                  max_length=max_length, return_tensors="pt").to(device)
        hs   = bert(**enc).last_hidden_state
        mask = enc.attention_mask.unsqueeze(-1)
        mean = (hs * mask).sum(1) / mask.sum(1).clamp(min=1)
        vecs.append(mean.cpu())
    return torch.cat(vecs, 0).numpy()

def emb_features(ds_lang):
    q = mean_embed(ds_lang["question"])
    c = mean_embed(ds_lang["context"])
    feats = np.concatenate([q, c], axis=1)
    return feats

hidden_size from config: 768



 Model 1: FFN on Mean-Embeddings (scores -> tuned threshold)

In [ ]:
class FFN(torch.nn.Module):
    def __init__(self, d=1536, h=128):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(d, h), torch.nn.ReLU(), torch.nn.Linear(h, 1)
        )
    def forward(self, x): return self.net(x).squeeze(-1)

def train_ffn_get_scores(Xtr, ytr, Xva, epochs=6, lr=1e-3, bs=64):
    Xtr_t = torch.tensor(Xtr, dtype=torch.float32)
    ytr_t = torch.tensor(ytr, dtype=torch.float32)
    net   = FFN(d=Xtr.shape[1]).to(device)
    # pos_weight = n_neg / n_pos
    n_pos, n_neg = ytr.sum(), len(ytr) - ytr.sum()
    pos_w = torch.tensor([ (n_neg / max(1, n_pos)) ], dtype=torch.float32).to(device)
    lossf = torch.nn.BCEWithLogitsLoss(pos_weight=pos_w)
    opt   = torch.optim.Adam(net.parameters(), lr=lr)

    net.train()
    for _ in range(epochs):
        idx = torch.randperm(len(Xtr_t))
        for i in range(0, len(Xtr_t), bs):
            b = idx[i:i+bs]
            xb, yb = Xtr_t[b].to(device), ytr_t[b].to(device)
            opt.zero_grad(); loss = lossf(net(xb), yb); loss.backward(); opt.step()

    net.eval(); scores = []
    with torch.no_grad():
        for i in range(0, len(Xva), 2048):
            xb = torch.tensor(Xva[i:i+2048], dtype=torch.float32).to(device)
            scores.append(torch.sigmoid(net(xb)).cpu().numpy())
    return np.concatenate(scores)


Model 2: RandomForest on Mean-Embeddings (prob -> tuned threshold)

In [ ]:
def rf_get_scores(Xtr, ytr, Xva):
    rf = RandomForestClassifier(n_estimators=300, random_state=0, n_jobs=-1, class_weight="balanced")  # more stable with imbalance
    rf.fit(Xtr, ytr)
    return rf.predict_proba(Xva)[:, 1]

Model 3: BoW + Logistic Regression (prob -> tuned)

In [ ]:
def fit_bow(train_lang, val_lang, max_features=20000):
    qv = TfidfVectorizer(max_features=max_features, ngram_range=(1,2))
    cv = TfidfVectorizer(max_features=max_features, ngram_range=(1,2))
    Xq_tr = qv.fit_transform(train_lang["question"])
    Xc_tr = cv.fit_transform(train_lang["context"])
    Xq_va = qv.transform(val_lang["question"])
    Xc_va = cv.transform(val_lang["context"])
    return hstack([Xq_tr, Xc_tr]), hstack([Xq_va, Xc_va])

def lr_get_scores(Xtr_bow, ytr, Xva_bow):
    lr = LogisticRegression(max_iter=1000, solver="liblinear", class_weight="balanced")
    lr.fit(Xtr_bow, ytr)
    return lr.predict_proba(Xva_bow)[:, 1]

 Running per language (Threshold tuning: goal="f1")

In [ ]:
for L in langs:
    trL = train_ds.filter(lambda ex: ex["lang"] == L)
    vaL = val_ds.filter(  lambda ex: ex["lang"] == L)
    y_tr, y_va = to_numpy_labels(trL), to_numpy_labels(vaL)
    print(f"\n=== {L} ===  train={len(trL)}  val={len(vaL)}")
    label_stats(y_tr, f"{L}-train"); label_stats(y_va, f"{L}-val")

    # embeddings
    Xtr_emb = emb_features(trL)
    Xva_emb = emb_features(vaL)
    print("Emb shape:", Xtr_emb.shape, Xva_emb.shape)

    # model 1: FFN-MeanEmb
    s_ffn = train_ffn_get_scores(Xtr_emb, y_tr, Xva_emb, epochs=6)
    thr1  = pick_threshold(y_va, s_ffn, goal="f1")
    y_ffn = (s_ffn >= thr1).astype(int)
    acc,tpr,fpr = metrics(y_va, y_ffn)
    print(f"[MODEL 1: FFN-MeanEmb]   thr={thr1:.2f}  Acc={acc:.3f}  TPR={tpr:.3f}  FPR={fpr:.3f}")

    # model 2: RF-MeanEmb
    s_rf = rf_get_scores(Xtr_emb, y_tr, Xva_emb)
    thr2 = pick_threshold(y_va, s_rf, goal="f1")
    y_rf = (s_rf >= thr2).astype(int)
    acc,tpr,fpr = metrics(y_va, y_rf)
    print(f"[MODEL 2: RF-MeanEmb]    thr={thr2:.2f}  Acc={acc:.3f}  TPR={tpr:.3f}  FPR={fpr:.3f}")

    # Model 3: BoW+LogReg
    Xtr_bow, Xva_bow = fit_bow(trL, vaL)
    s_lr = lr_get_scores(Xtr_bow, y_tr, Xva_bow)
    thr3 = pick_threshold(y_va, s_lr, goal="f1")
    y_lr = (s_lr >= thr3).astype(int)
    acc,tpr,fpr = metrics(y_va, y_lr)
    print(f"[MODEL 3: BoW+LogReg]    thr={thr3:.2f}  Acc={acc:.3f}  TPR={tpr:.3f}  FPR={fpr:.3f}")


Filter:   0%|          | 0/6335 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1155 [00:00<?, ? examples/s]


=== ar ===  train=2558  val=415
[ar-train] n=2558  positives=2303 (0.900)  negatives=0.100
[ar-val] n=415  positives=363 (0.875)  negatives=0.125
Emb shape: (2558, 1536) (415, 1536)
[MODEL 1: FFN-MeanEmb]   thr=0.34  Acc=0.973  TPR=0.978  FPR=0.058
[MODEL 2: RF-MeanEmb]    thr=0.70  Acc=0.940  TPR=0.997  FPR=0.462
[MODEL 3: BoW+LogReg]    thr=0.49  Acc=0.978  TPR=0.983  FPR=0.058


Filter:   0%|          | 0/6335 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1155 [00:00<?, ? examples/s]


=== ko ===  train=2422  val=356
[ko-train] n=2422  positives=2359 (0.974)  negatives=0.026
[ko-val] n=356  positives=337 (0.947)  negatives=0.053
Emb shape: (2422, 1536) (356, 1536)
[MODEL 1: FFN-MeanEmb]   thr=0.20  Acc=0.952  TPR=0.985  FPR=0.632
[MODEL 2: RF-MeanEmb]    thr=0.82  Acc=0.955  TPR=1.000  FPR=0.842
[MODEL 3: BoW+LogReg]    thr=0.65  Acc=0.955  TPR=0.988  FPR=0.632


Filter:   0%|          | 0/6335 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1155 [00:00<?, ? examples/s]


=== te ===  train=1355  val=384
[te-train] n=1355  positives=1310 (0.967)  negatives=0.033
[te-val] n=384  positives=291 (0.758)  negatives=0.242
Emb shape: (1355, 1536) (384, 1536)
[MODEL 1: FFN-MeanEmb]   thr=0.19  Acc=0.831  TPR=0.948  FPR=0.538
[MODEL 2: RF-MeanEmb]    thr=0.80  Acc=0.849  TPR=0.931  FPR=0.409
[MODEL 3: BoW+LogReg]    thr=0.56  Acc=0.812  TPR=0.866  FPR=0.355
